In [1]:
QTD_OF_SUMMARIES = 3
NR_EPOCHS = 1000
BATCH_SIZE = 10

In [2]:
import datautils.dataloader as loader

summaries = loader.load_summaries()

In [3]:
loader.load_mne_data(summaries[:QTD_OF_SUMMARIES])

Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Extracting EDF parameters from c:\Users\luiza\Desktop\Mestrado\Trabalho\data\chb01\chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)
c:\Users\luiza\Desktop\Mestrado\Trabalho\reader\mnereader.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  mne_model = mne.io.read_raw_edf(summary_model.fullpath(), include=commons.selected_channels(), preload=True)


In [4]:
loader.load_segmented_data(summaries[:QTD_OF_SUMMARIES])

### Data Preparation -> Dominio do Tempo 

- Segmentar as informações de tempo em janelas
- Criar as labels para cada janela
- Separar as informações em Treino e Teste
- Alimentar a rede e verificar a saída

In [5]:
import datautils.splitter.mnesplitter as mnesplitter

X_train, X_val, y_train, y_val = mnesplitter.segmented_time_data_splitter(summaries[:QTD_OF_SUMMARIES])

2160

In [11]:
len(y_train)

1440

In [8]:
import ia.model.cnn as ia

time_cnn = ia.CNNModel(input_shape=(X_train[0].shape[0], X_train[0].shape[1]))

In [9]:
time_cnn.compile()

In [10]:
time_cnn.fit(X_train, y_train, num_epochs=NR_EPOCHS, batch_size=BATCH_SIZE, val_data=X_val, val_labels=y_val)

ValueError: Data cardinality is ambiguous:
  x sizes: 4320
  y sizes: 1440
Make sure all arrays contain the same number of samples.

In [ ]:
# loader.load_mne_data(summaries[QTD_OF_SUMMARIES + 1:QTD_OF_SUMMARIES + 10])
# loader.load_segmented_data(summaries[QTD_OF_SUMMARIES + 1:QTD_OF_SUMMARIES + 10])

In [ ]:
# import numpy as np
# X_test = summaries[QTD_OF_SUMMARIES + 1 : QTD_OF_SUMMARIES + 10]
# y_test = [summary.has_anomaly() for summary in X_test]

# X_test_freq_data = np.array([summary.signal.get_freq_data()[0] for summary in X_test])

In [ ]:
# predictions = time_cnn.model.predict(X_test_freq_data)
# for i in range(len(y_test)):
#     print(str(predictions[i]) + ":" + str(y_test[i]))
# test_loss, test_acc = frequency_cnn.model.evaluate(X_test_freq_data, y_test)

# print(f'Test accuracy: {test_acc}')